In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

# Métricas
from sklearn.metrics import accuracy_score, recall_score, precision_score

# Este vai converter a última coluna de categorical data para integer
from sklearn.preprocessing import LabelEncoder

# Modelos
from sklearn.tree import DecisionTreeClassifier

# K-fold CrossValidation
from sklearn.model_selection import KFold

In [ ]:
from zipfile import ZipFile
pasta_geral = '/content/sonaralldata.zip'
zip_file = ZipFile(pasta_geral)

In [ ]:
# Load the CSV file into a pandas DataFrame
sonar_df = pd.read_csv(zip_file.open('sonar.all-data.csv'), header=None)

In [ ]:
X = sonar_df.iloc[:, :-1]
y_class = sonar_df[60]

In [ ]:
# Converter última coluna usando LabelEncoder()
le = LabelEncoder()
y = le.fit_transform(y_class)

In [ ]:
# Alterar os valores 0 para -1
for i in range(len(y)):
    if y[i] == 0:
        y[i] = -1
y

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1])

## O nosso algoritmo_AdaBoost

In [ ]:
def algoritmo_AdaBoost(X_train, y_train, M=10, random_state=1):
    
    #Initialização - distribuição de pesos
    
    # Número de exemplos rotulados (total de registros para treino)
    N = len(y_train)
    # Ciclo atual de aprendizado
    t = np.ones(N) # retorna um array de números 1
    # Distribuição de pesos atríbuída aos exemplos de treinamento
    D = t / N
    
    list_clf, y_list_pred, list_erro_clf, list_pesos_clf, list_pesos_amostra = [], [],[],[],[]    
    list_pesos_amostra.append(D.copy())

    # Número total de iterações
    T = range(M)
    
    # Treino do algoritmo base utilizando a distribuição D
    for n in T:   

        print('____________________________________________________________________________')
        print('Iteração: {}'.format(n + 1))
        
        sample_X_train, sample_y_train = resample(X_train, 
                                                  y_train, 
                                                  n_samples=N,
                                                  random_state=random_state)
        
        clf = DecisionTreeClassifier(max_depth = 10, min_samples_leaf=7)
        
        print('Após reamostragem....')
        print('Conjunto de treinamento - Dados {} - {}'.format(sample_X_train.shape, sample_y_train.shape))
        
        # Treino de cada modelo
        clf.fit(sample_X_train, sample_y_train, sample_weight=D)
        
        y_pred = clf.predict(sample_X_train)
        # Retorna a Hipótese H
        c = len(y_train)
        H = (y_pred != y_train)
        H_total = H.sum()
        print('H Total:{}'.format(H_total))
                
        # Cálculo do erro da hipótese H
        E = np.mean(np.average(H, weights=D, axis=0))
        print('E:{}'.format(E))
        
        # Importância associada do classificador, calculada a partir do erro E
        iclf =  0.5 * np.log((1 - E) / E)
        print('iclf:{}'.format(iclf))
        
        # Cálculo dos pesos
        w = D[n]
        print('w:{}'.format(w))
        
        corretos = w * np.exp(-iclf)
        incorretos = w * np.exp(iclf)
        
        print('corretos:{}'.format(corretos))
        print('incorretos:{}'.format(incorretos))
        
        # Termo normalizador
        Z = (corretos * (c-H_total)) + (incorretos * H_total)
        print('Z:{}'.format(Z))
        
        corr_normal = corretos / Z
        incorr_normal = incorretos / Z
        print('corretos normalizado:{}'.format(corr_normal))
        print('incorretos normalizado:{}'.format(incorr_normal))
        
        teste_Z = (corr_normal * (c-H_total)) + (incorr_normal * H_total)
        print('teste normalização:{}'.format(teste_Z))
        
        # Atualização de D
        for m in range(c):
            if (y_pred[m] != y_train[m]):
                D[m] = (D[m] / Z) * np.exp(iclf)
            else:
                D[m] = (D[m] / Z) * np.exp(-iclf)
        
        print('D:{}'.format(D))

        # Guarda valores da iteração
        #list_clf.append(clf)
        y_list_pred.append(y_pred.copy())
        list_pesos_clf.append(iclf.copy())
        
    #Converte para np array   
    #list_clf = np.asarray(list_clf)
    y_list_pred = np.asarray(y_list_pred)
    list_pesos_clf = np.asarray(list_pesos_clf)
    
    print('____________________________________________________________________________')
    
    # Cálculo de métricas
    # HIPÓTESE FINAL        
    preds = (np.array([np.sign((y_list_pred[:,point] * list_pesos_clf).sum()) for point in range(N)]))
    #print('preds:{}'.format(preds))
    
    acuracia = accuracy_score(y_train, preds)
    recall = recall_score(y_train, preds)
    precisao = precision_score(y_train, preds)
    
    # COMPARA O RESULTADO DO MODELO COM OS RESULTADOS DO TESTE
    print('Acurácia: {}'.format(acuracia))
    print('Recall: {}'.format(recall))
    print('Precisão: {}'.format(precisao))
    print('____________________________________________________________________________')    
    
    return acuracia, recall, precisao, clf

# Cross Validation

In [ ]:
n_modelos = 10
kf = KFold(n_splits=10)
random_state=42

curr_fold = 1
acuracia_total = 0
recall_total = 0
precisao_total = 0    

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=1)

# Converte X para NumPy Array
X_np = X.values

for train_index, test_index in kf.split(X_np):
    
    X_train, X_test = X_np[train_index], X_np[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    print('Validação Cruzada - Fold {}'.format(curr_fold))
    print('Conjunto de treinamento - Dados {} - {}'.format(X_train.shape, y_train.shape))
    print('Conjunto de teste - Dados {} - {}'.format(X_test.shape, y_test.shape))
    
    acuracia, recall, precisao, clf  = algoritmo_AdaBoost(X_train, y_train, M=n_modelos, random_state=random_state)
    
    acuracia_total += acuracia
    recall_total += recall
    precisao_total += precisao
    
    curr_fold += 1
    
print('############################################################################')    
print('Resultado Treino')    
print('Acurácia FINAL: {}'.format(acuracia_total/(curr_fold-1)))
print('Recall FINAL: {}'.format(recall_total/(curr_fold-1)))
print('Precisão FINAL: {}'.format(precisao_total/(curr_fold-1)))
print('KFolds: {}'.format(curr_fold-1))

Validação Cruzada - Fold 1
Conjunto de treinamento - Dados (187, 60) - (187,)
Conjunto de teste - Dados (21, 60) - (21,)
____________________________________________________________________________
Iteração: 1
Após reamostragem....
Conjunto de treinamento - Dados (187, 60) - (187,)
H Total:90
E:0.48128342245989303
iclf:0.037450654086558954
w:0.0053475935828877
corretos:0.005151026463111782
incorretos:0.005551661854687143
Z:0.9992991338436858
corretos normalizado:0.005154639175257731
incorretos normalizado:0.005555555555555555
teste normalização:0.9999999999999998
D:[0.00555556 0.00555556 0.00555556 0.00515464 0.00555556 0.00515464
 0.00515464 0.00555556 0.00555556 0.00515464 0.00555556 0.00555556
 0.00555556 0.00555556 0.00555556 0.00555556 0.00555556 0.00515464
 0.00515464 0.00555556 0.00515464 0.00515464 0.00555556 0.00515464
 0.00555556 0.00515464 0.00515464 0.00555556 0.00515464 0.00555556
 0.00555556 0.00515464 0.00555556 0.00515464 0.00555556 0.00555556
 0.00515464 0.00555556 0.0

In [ ]:
print('############################################################################')    
print('Teste...')
y_predict = clf.predict(X_test)
print('Predição         : {}'.format(y_predict))
print('Amostra de Teste : {}'.format(y_test))
print('Comparação       : {}'.format(y_predict == y_test))
acuracia_test = accuracy_score(y_test, y_predict)
print('Acurácia do Teste: {}'.format(acuracia_test))

############################################################################
Teste...
Predição         : [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1  1]
Amostra de Teste : [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
Comparação       : [ True  True  True  True  True  True  True  True  True  True  True  True
  True False  True  True  True  True  True False]
Acurácia do Teste: 0.9
